In [2]:
import os

In [3]:
%pwd

'/Users/varundesai/Visual_Studio_Varun/Projects/Text Summariser Project/notebook'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/Users/varundesai/Visual_Studio_Varun/Projects/Text Summariser Project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [7]:
from textSummarizer.constant import *
from textSummarizer.utils.common import create_directories, read_yaml


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! With following headers: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def unzip_data(self):
        unzip_path  = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try:
    config  = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.unzip_data()
except Exception as e:
    raise e

[2024-02-20 14:26:50,382: INFO: common yaml file: config/config.yaml loaded successfully]
[2024-02-20 14:26:50,384: INFO: common yaml file: params.yaml loaded successfully]
[2024-02-20 14:26:50,384: INFO: common Created directory at: artifacts]
[2024-02-20 14:26:50,385: INFO: common Created directory at: artifacts/data_ingestion]
[2024-02-20 14:26:57,377: INFO: 4058958440 artifacts/data_ingestion/data.zip download! With following headers: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 478A:3E84F9:1ED80E:21DD0F:65D46953
Accept-Ranges: bytes
Date: Tue, 20 Feb 2024 08:56:52 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4732-BOM
X-Cache: MISS